In [3]:
queryDic = {}
file = open('/qbio/prom317/tutorial/transCutAlignment/CLIP_35L33G/Aligned.out.sam', 'r')
lines = file.readlines()
for line in lines:
    if line.startswith("SRR"):
        if "ENSMUST00000083506.3" in line:
            lineSep = line.split('\t')
            queryName = lineSep[0]
            pos = int(lineSep[3])
            cigar = lineSep[5]
            queryDic[queryName] = [pos, cigar]

In [10]:
import re
p = re.compile("[0-9]+M")
for i in queryDic.keys():
    cigar = queryDic[i][1]
    m=p.search(cigar)
    if m:
        newCigar = m.group()
        queryDic[i].append(newCigar) 

In [8]:
baseMatchDic = {}
for i in range(1, 89):
    baseMatchDic[i] = [0,0]

refseq = "CCAGGCTGAGGTAGTAGTTTGTACAGTTTGAGGGTCTATGATACCACCCGGTACAGGAGATAACTGTACAGGCCACTGCCTTGCCAGG"
for i in queryDic.keys():
    

'65M'